<a href="https://colab.research.google.com/github/jamesBaker361/colab/blob/master/toot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
import numpy as np

tf.logging.set_verbosity(tf.logging.INFO)

https://www.tensorflow.org/tutorials/estimators/cnn

In [0]:
def cnn_model_fn(features, labels, mode):
  input_layer = tf.reshape(features["x"], [-1, 32, 32, 1])
  conv1 = tf.layers.conv2d(
    inputs=input_layer,
    filters=64,
    kernel_size=[3, 3],
    padding="same",
    activation=tf.nn.relu)
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[1, 1],strides=1)
  conv2 = tf.layers.conv2d(
    inputs=pool1,
    filters=128,
    kernel_size=[3, 3],
    padding="same",
    activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[1, 1],strides=1)
  conv3 = tf.layers.conv2d(
    inputs=pool2,
    filters=256,
    kernel_size=[3, 3],
    padding="same",
    activation=tf.nn.relu)
  conv4 = tf.layers.conv2d(
    inputs=conv3,
    filters=256,
    kernel_size=[3, 3],
    padding="same",
    activation=tf.nn.relu)
  pool4=tf.layers.max_pooling2d(inputs=conv4, pool_size=[1, 1],strides=1)
  conv5 = tf.layers.conv2d(
    inputs=pool4,
    filters=512,
    kernel_size=[3, 3],
    padding="same",
    activation=tf.nn.relu)
  conv6 = tf.layers.conv2d(
    inputs=conv5,
    filters=512,
    kernel_size=[3, 3],
    padding="same",
    activation=tf.nn.relu)
  pool6=tf.layers.max_pooling2d(inputs=conv6, pool_size=[1, 1],strides=1)
  conv7 = tf.layers.conv2d(
    inputs=pool6,
    filters=512,
    kernel_size=[3, 3],
    padding="same",
    activation=tf.nn.relu)
  conv8 = tf.layers.conv2d(
    inputs=conv7,
    filters=512,
    kernel_size=[3, 3],
    padding="same",
    activation=tf.nn.relu)
  pool8=tf.layers.max_pooling2d(inputs=conv8, pool_size=[1, 1], strides=1)
  pool8flat=tf.reshape(pool8,[-1,1])
  dense1 = tf.layers.dense(inputs=pool8flat, units=4096, activation=tf.nn.relu)
  dense2 = tf.layers.dense(inputs=dense1, units=4096, activation=tf.nn.relu)
  dense3=tf.layers.dense(inputs=dense2, units=1000, activation=tf.nn.relu)
  dropout = tf.layers.dropout(inputs=dense3, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
  logits = tf.layers.dense(inputs=dropout, units=100)
  predictions = {
    "classes": tf.argmax(input=logits, axis=1),
    "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
  loss = tf.losses.softmax_cross_entropy(labels=labels, logits=logits)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
      loss=loss,
      global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])
  }
  return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [0]:
((x_train, y_train), (x_test, y_test))=tf.keras.datasets.cifar10.load_data()
#((train_data, train_labels),(eval_data, eval_labels))

In [0]:
x_train = x_train/np.float32(255)
y_train = y_train.astype(np.int32)  # not required

x_test = x_test/np.float32(255)
y_test = y_test.astype(np.int32)  # not required

In [0]:
classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="/tmp/cifar_convnet_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/cifar_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9569253da0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
tensors_to_log = {"probabilities": "softmax_tensor"}

logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

In [0]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

In [0]:
classifier.train(
    input_fn=train_input_fn,
    steps=1)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


TypeError: ignored